In [1]:
import numpy as np
from numpy.random import randn

In [2]:
n_y = 10
n_s = 12
hid = 64
batch = 256
h = randn(batch, hid)
W = randn(batch, hid, n_s)
X = randn(batch, n_s, n_y, hid)

In [3]:
print(np.einsum_path('bsyh,bhs,bh->by', X, W, h, optimize='optimal')[1])

  Complete contraction:  bsyh,bhs,bh->by
         Naive scaling:  4
     Optimized scaling:  4
      Naive FLOP count:  5.898e+06
  Optimized FLOP count:  4.129e+06
   Theoretical speedup:  1.429
  Largest intermediate:  1.966e+05 elements
--------------------------------------------------------------------------
scaling                  current                                remaining
--------------------------------------------------------------------------
   3                 bh,bhs->shb                             bsyh,shb->by
   4                shb,bsyh->by                                   by->by


In [25]:
def opt(X, W, h):
    tmp = np.einsum('bh,bhs->bhs', h, W)
    return np.einsum('bhs,bsyh->by', tmp, X)

In [27]:
%timeit -r 16 -n 100 np.einsum('bsyh,bhs,bh->by', X, W, h, optimize='optimal')
%timeit -r 16 -n 100 opt(X, W, h)

17.9 ms ± 861 µs per loop (mean ± std. dev. of 16 runs, 100 loops each)
17.2 ms ± 889 µs per loop (mean ± std. dev. of 16 runs, 100 loops each)


In [28]:
opt(X, W, h).shape

(256, 10)